# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df=pd.read_csv('../output_data/cities.csv')
weather_df.head()

,City,Latitude,Longitude,Max Temp(F),Humidity(%),Cloudiness(%),Wind Speed (mph),Country,Date
0,Cape Town,-33.9258,18.4232,77.11,70,0,14.97,ZA,1666192968
1,Mukhen,48.1000,136.1000,37.36,65,55,8.59,RU,1666192969
2,Pevek,69.7008,170.3133,26.37,96,100,7.11,RU,1666192969
3,Pontes e Lacerda,-15.2261,-59.3353,95.34,39,83,3.74,BR,1666192970
4,Puerto Ayora,-0.7393,-90.3518,73.36,89,86,5.99,EC,1666192970


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Pass the api key to the Google Maps API
gmaps.configure(api_key=g_key)

# store the coordinates from the DataFrame into a list (for passing longitude and latitude to gmaps)
coord = weather_df[['Latitude', 'Longitude']]

# story the humidity values from the DataFrame into a list (for passing to gmaps for the heatmap value)
humidity = weather_df['Humidity(%)'].astype(float)

In [4]:
# Create the gmaps figure & settings
fig = gmaps.figure(zoom_level=1, center=[0,0], map_type='TERRAIN', layout={'width': '600px', 'height': '500px'})

# Generate the heatmap layer based on the humidity
heat_layer = gmaps.heatmap_layer(coord, weights=humidity, 
                                 dissipating=False, max_intensity=100, point_radius=3)

# Add the heatmap layer to the figure
fig.add_layer(heat_layer)

# display the gmaps figure
fig

Figure(layout=FigureLayout(height='500px', width='600px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#My choice of ideal weather is max temperature between 65 & 75, wind speed <10 and little cloudiness
clean_df = weather_df.loc[(weather_df['Wind Speed (mph)'] >0) & (weather_df['Wind Speed (mph)'] < 10) & 
                       (weather_df['Max Temp(F)'] >= 65) & (weather_df['Max Temp(F)'] < 75) & 
                       (weather_df['Cloudiness(%)'] < 5)]

# Display the final dataframe
clean_df

#there are no null values in my dataframe, so nothing to drop

,City,Latitude,Longitude,Max Temp(F),Humidity(%),Cloudiness(%),Wind Speed (mph),Country,Date
8,Kahului,20.8947,-156.4700,73.85,73,0,4.61,US,1666192972
18,Guerrero Negro,27.9769,-114.0611,71.47,71,1,5.12,MX,1666192978
20,Laguna,38.4210,-121.4238,68.18,83,0,3.44,US,1666192983
63,Tezu,27.9167,96.1667,68.88,78,0,4.16,IN,1666193007
211,Port Hedland,-20.3167,118.5667,71.65,56,0,5.75,AU,1666193074
234,Corinth,37.9407,22.9573,70.90,35,0,1.43,GR,1666193081
248,Mount Isa,-20.7333,139.5000,74.97,64,0,6.91,AU,1666193087
254,Makakilo City,21.3469,-158.0858,71.98,81,0,5.75,US,1666193091
286,Sirte,31.2089,16.5887,73.36,55,1,9.15,LY,1666193102
343,Palera,25.0167,79.2333,73.63,57,0,4.59,IN,1666193122


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df= clean_df
# hotel_df["Hotel Name"] = ""

In [7]:
# define the base URL
base_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

# Define the required parameters to send with the API request
params = {'radius': 5000,
         'type': 'hotel',
         'key': g_key}

for index, row in hotel_df.iterrows():
    # get latitude, longitude from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
          
    response = requests.get(base_url, params=params).json()
          
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[1]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[1]['name']
              
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

clean_df

Retrieving Results for Index 8: Kahului.
Closest hotel is Maui Seaside Hotel.
------------
Retrieving Results for Index 18: Guerrero Negro.


C:\Users\lbhimavarapu\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\lbhimavarapu\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Closest hotel is COMERBAJA.
------------
Retrieving Results for Index 20: Laguna.
Closest hotel is Holiday Inn Express & Suites Elk Grove Central - Hwy 99, an IHG Hotel.
------------
Retrieving Results for Index 63: Tezu.
Closest hotel is Consumer Court.
------------
Retrieving Results for Index 211: Port Hedland.
Closest hotel is The Esplanade Hotel.
------------
Retrieving Results for Index 234: Corinth.
Closest hotel is Archeological site of Isthmia.
------------
Retrieving Results for Index 248: Mount Isa.
Closest hotel is ibis Styles Mt Isa Verona.
------------
Retrieving Results for Index 254: Makakilo City.
Closest hotel is Wet 'n' Wild Hawaii.
------------
Retrieving Results for Index 286: Sirte.
Closest hotel is Sirte Central.
------------
Retrieving Results for Index 343: Palera.
Closest hotel is Maa Ambe Cyber Cafe (MACC).
------------
Retrieving Results for Index 379: Faya.
Closest hotel is صراف الإنماء.
------------
Retrieving Results for Index 395: Shājāpur.
Closest hotel

,City,Latitude,Longitude,Max Temp(F),Humidity(%),Cloudiness(%),Wind Speed (mph),Country,Date,Hotel Name
8,Kahului,20.8947,-156.4700,73.85,73,0,4.61,US,1666192972,Maui Seaside Hotel
18,Guerrero Negro,27.9769,-114.0611,71.47,71,1,5.12,MX,1666192978,COMERBAJA
20,Laguna,38.4210,-121.4238,68.18,83,0,3.44,US,1666192983,Holiday Inn Express & Suites Elk Grove Central...
63,Tezu,27.9167,96.1667,68.88,78,0,4.16,IN,1666193007,Consumer Court
211,Port Hedland,-20.3167,118.5667,71.65,56,0,5.75,AU,1666193074,The Esplanade Hotel
234,Corinth,37.9407,22.9573,70.90,35,0,1.43,GR,1666193081,Archeological site of Isthmia
248,Mount Isa,-20.7333,139.5000,74.97,64,0,6.91,AU,1666193087,ibis Styles Mt Isa Verona
254,Makakilo City,21.3469,-158.0858,71.98,81,0,5.75,US,1666193091,Wet 'n' Wild Hawaii
286,Sirte,31.2089,16.5887,73.36,55,1,9.15,LY,1666193102,Sirte Central
343,Palera,25.0167,79.2333,73.63,57,0,4.59,IN,1666193122,Maa Ambe Cyber Cafe (MACC)


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='500px', width='600px'))